In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython 

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 102kB 4.1MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model

In [8]:
tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="intro_to_kt",
)

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_End(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [10]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.852400004863739

Best val_accuracy So Far: 0.8849999904632568
Total elapsed time: 00h 18m 57s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4815 - accuracy: 0.8306 - val_loss: 0.4491 - val_accuracy: 0.8397
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3644 - accuracy: 0.8679 - val_loss: 0.3751 - val_accuracy: 0.8635
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3274 - accuracy: 0.8792 - val_loss: 0.3572 - val_accuracy: 0.8732
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3042 - accuracy: 0.8880 - val_loss: 0.3433 - val_accuracy: 0.8773
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2844 - accuracy: 0.8940 - val_loss: 0.3401 - val_accuracy: 0.8782
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2697 - accuracy: 0.8999 - val_loss: 0.3322 - val_accuracy: 0.8772
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2567 - accuracy: 0.9029 - val_loss: 0.3285 - val_accuracy: